<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import torch, torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd
from IPython.display import display
from pathlib import Path

"""
Скрипт сравнения распределений P(next_token) с поддержкой:
----------------------------------------------------------------
1. **Внутренних промптов** (список в коде) **или**
2. **Промптов из текстового файла** в той же директории.

Флаг `USE_FILE_PROMPTS` выбирает источник.
Если `True`, файл `PROMPT_FILE` читается целиком и используется как **один** длинный prompt.

Модификация «summary + слова» печатает краткое summary исходного prompt‑а.
Добавлена **безопасная токенизация**: если длиннее лимита модели (1024 для GPT‑2),
текст автоматически усечётся, а в консоль выведется предупреждение.
"""

# ---------- ПАРАМЕТРЫ ----------
MODEL_NAME   = "gpt2"
STEPS        = 3
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

# Флаг, управляющий включением суммаризации как отдельной модификации.
# Если PROMPTS короткие или суммаризация не нужна, установите False.
SUMMARY_ENABLED = True  # ← меняйте по необходимости

# Суммаризация
SUMM_MODEL   = "sshleifer/distilbart-cnn-12-6"
CUSTOM_WORDS = "Сводка:"

# Выбор источника промптов
USE_FILE_PROMPTS = True             # ← переключите на True для чтения из файла
PROMPT_FILE      = "prompt.txt"      # UTF‑8 файл в той же папке

# ---------- 1. Модели ----------
_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
_model     = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE).eval()

_summarizer = pipeline(
    "summarization",
    model=SUMM_MODEL,
    tokenizer=SUMM_MODEL,
    device=0 if DEVICE=="cuda" else -1,
)

CTX_LIMIT = _model.config.n_positions  # 1024 для GPT‑2

# ---------- 2. Вспомогательные функции ----------

def _next_token_probs(ids):
    """Вернёт софтмакс‑распределение для следующего токена."""
    with torch.no_grad():
        logits = _model(ids).logits[0, -1]
    return torch.softmax(logits, dim=-1).cpu()


def _kl(p, q):
    return F.kl_div(p.log(), q, reduction="batchmean").item()


def _cos(p, q):
    return F.cosine_similarity(p, q, dim=0).item()


def _multi_step(prompt: str, n: int):
    """Возвращает список распределений для n последующих токенов.
    Если prompt длиннее контекст‑лимита, автоматически усечёт и предупредит."""
    token_ids = _tokenizer.encode(prompt)
    if len(token_ids) > CTX_LIMIT:
        print(f"⚠️ Промпт содержит {len(token_ids)} токенов и будет усечён до {CTX_LIMIT}.")
    ids = _tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=CTX_LIMIT
        )["input_ids"].to(DEVICE)

    dists = []
    for _ in range(n):
        probs = _next_token_probs(ids)
        dists.append(probs)
        next_id = probs.argmax().unsqueeze(0).unsqueeze(0).to(DEVICE)
        ids = torch.cat([ids, next_id], dim=1)
    return dists

# ---------- 3. Источник промптов ----------
if USE_FILE_PROMPTS:
    path = Path(PROMPT_FILE)
    if not path.is_file():
        raise FileNotFoundError(f"Файл {PROMPT_FILE} не найден")
    PROMPTS = [path.read_text(encoding="utf-8").strip()]
else:
    PROMPTS = [
        "Why the stock market is expected to",
        "The future of artificial intelligence depends on",
    ]
    

# ---------- 4. Модификации ----------
def add_typo_first_e(p: str) -> str:
    """Replace the first occurrence of 'e' with '3'."""
    return p.replace("e", "3", 1)

def add_salutation(p: str) -> str:
    """Prepend a greeting line."""
    return "Dear GPT,\n" + p


MODS = {
    "префикс ======":   lambda p: "="*10 + p,
     "префикс question(англ)":  lambda p: "I have a question. " + p,
    "префикс вопрос":  lambda p: "У меня есть вопрос. " + p,
    "суффикс 10 лет?": lambda p: p + " in the next decade?",
}

# --- Модификация с суммаризацией и печатью ---

def summary_mod(custom: str = CUSTOM_WORDS):
    """Возвращает модификатор, который:
    ▸ один раз печатает summary для каждого уникального промпта (до расчётов),
    ▸ кэширует результат, чтобы в блоке 5 печать не дублировалась,
    ▸ добавляет summary в начало текста.
    """
    cache = {}

    def _fn(p: str):
        if p not in cache:  # печатаем только при первом появлении промпта
            summary = _summarizer(p, max_length=60, min_length=15, do_sample=False)[0]["summary_text"]
            print(f"\n[SUMMARY]\n{summary}\n")
            cache[p] = summary
        else:
            summary = cache[p]
        return f"{custom} {summary}\n\n{p}"

    return _fn

# создаём функцию‑модификатор и сразу кладём её в словарь
if SUMMARY_ENABLED:
    summary_fn = summary_mod()
    MODS["summary + слова (print)"] = summary_fn

 # ---------- 4.1. Предварительный вывод Summary (до расчётов) ----------
    for _p in PROMPTS:
        summary_fn(_p)
# пройдёмся по всем промптам и вызовем summary_fn, чтобы вывести summary ЗАРАНЕЕ



C:\Users\cab\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\cab\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (



[SUMMARY]
 Every morning, Sarah walked the same path to the train station, passing the same bakery, the same newsstand, and the same park bench . An old man sat there, feeding the birds, and nodded to her as she passed . Slowly, their nods became smiles, then greetings



In [2]:

# ---------- 5. Расчёт ----------
records = []
for prompt in PROMPTS:
    base = _multi_step(prompt, STEPS)
    for mod_name, mod_fn in MODS.items():
        mod  = _multi_step(mod_fn(prompt), STEPS)
        for step in range(STEPS):
            records.append({
                "prompt": prompt[:80] + ("…" if len(prompt) > 80 else ""),
                "mod":    mod_name,
                "step":   step + 1,
                "cos":    _cos(base[step], mod[step]),
                "kl":     _kl(base[step],  mod[step]),
            })

_df = pd.DataFrame(records)
pd.set_option("display.precision", 4)




In [3]:
# ---------- 6. Вывод ----------  
    
# ---------- 6.1 Вывод Cosine----------
for s in range(1, STEPS + 1):
    df = _df[_df.step == s]
    print(f"\n=== Шаг {s} — Cosine ===")
    display(df.pivot(index="prompt", columns="mod", values="cos"))
    #print(f"\n=== Шаг {s} — KL ===")
    #display(df.pivot(index="prompt", columns="mod", values="kl"))
    
# ---------- 6.2 Вывод KL----------
for s in range(1, STEPS + 1):
    df = _df[_df.step == s]
    #print(f"\n=== Шаг {s} — Cosine ===")
    #display(df.pivot(index="prompt", columns="mod", values="cos"))
    print(f"\n=== Шаг {s} — KL ===")
    display(df.pivot(index="prompt", columns="mod", values="kl"))

# ---------- 7. Лимит строк (без ошибки в Jupyter) ----------
import inspect, sys
try:
    if len(inspect.getsource(sys.modules[__name__]).splitlines()) > 175:
        print("⚠️ Файл приближается к лимиту, рассмотрите рефакторинг.")
except (TypeError, OSError):
    pass



=== Шаг 1 — Cosine ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
"Every morning, Sarah walked the same path to the train station, passing the same…",0.9634,0.9561,0.9957,0.9764,0.581



=== Шаг 2 — Cosine ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
"Every morning, Sarah walked the same path to the train station, passing the same…",1.0,0.0006,1.0,0.0006,9.7080e-06



=== Шаг 3 — Cosine ===


...

mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
"Every morning, Sarah walked the same path to the train station, passing the same…",8.2318e-07,8.0658e-07,9.7220e-08,3.2189e-07,3.7918e-05



=== Шаг 2 — KL ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
"Every morning, Sarah walked the same path to the train station, passing the same…",8.6047e-08,0.0004,5.7652e-10,0.0004,0.0005



=== Шаг 3 — KL ===


mod,summary + слова (print),префикс ======,префикс question(англ),префикс вопрос,суффикс 10 лет?
prompt,,,,,
"Every morning, Sarah walked the same path to the train station, passing the same…",3.3205e-06,8.3405e-05,1.3679e-07,7.6004e-05,0.0001
